In [ ]:
## Arxiv-- Research
## Tools creation

from langchain.tools import ArxivQueryRun,WikipediaQueryRun  ## # Import two ready-made tools from LangChain
# These tools let your program ASK questions to Arxiv and Wikipedia
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper ## Import helper classes that actually know HOW to talk to Arxiv and Wikipedia
# These handle API calls, formatting, and fetching results

In [ ]:
### Create an object that knows how to fetch data from Wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(
    top_k_results=1,# Get only 1 best search result
    doc_content_chars_max=250 # Limit article text to 250 characters
)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki) # Create a Wikipedia search tool using the above wrapper
wiki.name # Get the internal name of the Wikipedia tool (used by LangChain agents to identify it)


'wikipedia'

In [ ]:
# Create an object that knows how to fetch research papers from Arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv) # Create an Arxiv search tool using the above wrapper
arxiv.name  ##Get the internal name of the Arxiv tool (used by LangChain)


'arxiv'

In [ ]:
# Put both tools into a list
# This list is later given to an AI agent or chain
# So the AI knows which tools it is allowed to use
tools=[wiki,arxiv]

In [7]:
## Create Custom Tools [RAG Tool] 
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

import os
from dotenv import load_dotenv
load_dotenv()

os.environ['HF_TOKEN']=os.getenv("HF_TOKEN") ## Load HF token from .env file
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Artificial_intelligence")
docs=loader.load()
document=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(document,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x303c23ed0>, search_kwargs={})

In [ ]:
# Import a helper function that converts a retriever into a usable tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"AISearch","Search about AI on Wikipedia") # The retriever that fetches documents,AISearch is the name of the tool, description is instruction to the AI
retriever_tool.name # Get the internal name of the retriever tool

'AISearch'

In [ ]:
tools=[wiki,arxiv,retriever_tool] ### Create a list that contains all the tools you want your AI to use
tools # Just display or check what tools are inside the list

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/anaconda3/lib/python3.13/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='AISearch', description='Search about AI on Wikipedia', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x31d073600>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x303c23ed0>, search_kwargs={}), document_prompt=PromptTe

In [12]:
## Run all this tool with Agent and LLM model

## Tools, LLM--> AgentExecutor --> Final Answer

import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(api_key=api_key,model_name="llama-3.1-8b-instant")

In [ ]:
## Prompt Template LangChain Hub is like a store of ready-made prompts.These prompts are written by experts.You don’t have to design everything from scratch
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent") # Download a predefined agent prompt from the LangChain Hub.It tells ai how to think,choose tool and respond
prompt.messages # View the messages that make up this prompt

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
## Agents

from langchain.agents import create_openai_tools_agent ##This function knows how to build an AI agent using OpenAI function-calling style
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [ ]:
## Agent Executor

from langchain.agents import AgentExecutor #This class is responsible for running the agent
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True) # The agent you created earlier (the brain),# Tools the agent is allowed to use, # Show step-by-step thinking and tool usage in logs
agent_executor #Display the executor object

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [18]:
agent_executor.invoke({"input":"Explain artificial intelligence."})



> Entering new AgentExecutor chain...

Invoking: `AISearch` with `{'query': 'artificial intelligence'}`


Glossary
Glossary
vte
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]

Edit links











ArticleTalk





English

















ReadView sourceView history







Tools





Tools
move to sidebar
hide



		Actions
	


ReadView sourceView history





		General
	


What links hereRelated changesUpload filePermanent linkPage informationCite this pageGet shortened URLDownload QR code





		Print/export
	


Download as PDFPrintable version





		In other projects


{'input': 'Explain artificial intelligence.',
 'output': 'Agent stopped due to max iterations.'}

In [19]:
agent_executor.invoke({"input":"What's the research paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe research paper 1706.03762 is titled "Attention Is All You Need" and was published in 2017. It is about a new approach to sequence transduction models, using self-attention and multi-head attention mechanisms.

> Finished chain.


{'input': "What's the research paper 1706.03762 about?",
 'output': 'The research paper 1706.03762 is titled "Attention Is All You Need" and was published in 2017. It is about a new approach to sequence transduction models, using self-attention and multi-head attention mechanisms.'}